<a href="https://colab.research.google.com/github/jeffking1998/pytorch_RS_D2L/blob/main/NeuCF_GFM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
# tf.debugging.set_log_device_placement(True)

Num GPUs Available:  1


In [ ]:
## connect Google Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True) #, force_remount=True
mvlens_dir = '/content/drive/MyDrive/data/ml-1m/'

Mounted at /content/drive


In [ ]:
!ls /content/drive/MyDrive/data/ml-1m/

checkpoint			     ml-1m.test.rating
gmf.ckpt.data-00000-of-00001	     ml-1m.train.rating
gmf.ckpt.index			     pinterest-20.test.negative
gmf_on_gpu.ckpt.data-00000-of-00001  pinterest-20.test.rating
gmf_on_gpu.ckpt.index		     pinterest-20.train.rating
ml-1m.test.negative


In [ ]:
import scipy.sparse as sp
import numpy as np
import multiprocessing
import heapq
import os
import math 
from time import time

In [ ]:

class Dataset(object):
    '''
    classdocs
    '''

    def __init__(self, path):
        '''
        Constructor
        '''
        self.trainMatrix = self.load_rating_file_as_matrix(path + ".train.rating")
        self.testRatings = self.load_rating_file_as_list(path + ".test.rating")
        self.testNegatives = self.load_negative_file(path + ".test.negative")
        assert len(self.testRatings) == len(self.testNegatives)
        
        self.num_users, self.num_items = self.trainMatrix.shape
        
    def load_rating_file_as_list(self, filename):
        ratingList = []
        with open(filename, "r") as f:
            line = f.readline()
            while line != None and line != "":
                arr = line.split("\t")
                user, item = int(arr[0]), int(arr[1])
                ratingList.append([user, item])
                line = f.readline()
        return ratingList
    
    def load_negative_file(self, filename):
        negativeList = []
        with open(filename, "r") as f:
            line = f.readline()
            while line != None and line != "":
                arr = line.split("\t")
                negatives = []
                for x in arr[1: ]:
                    negatives.append(int(x))
                negativeList.append(negatives)
                line = f.readline()
        return negativeList
    
    def load_rating_file_as_matrix(self, filename):
        '''
        Read .rating file and Return dok matrix.
        The first line of .rating file is: num_users\t num_items
        '''
        # Get number of users and items
        num_users, num_items = 0, 0
        with open(filename, "r") as f:
            line = f.readline()
            while line != None and line != "":
                arr = line.split("\t")
                u, i = int(arr[0]), int(arr[1])
                num_users = max(num_users, u)
                num_items = max(num_items, i)
                line = f.readline()
        # Construct matrix
        mat = sp.dok_matrix((num_users+1, num_items+1), dtype=np.float32)
        with open(filename, "r") as f:
            line = f.readline()
            while line != None and line != "":
                arr = line.split("\t")
                user, item, rating = int(arr[0]), int(arr[1]), float(arr[2])
                if (rating > 0):
                    mat[user, item] = 1.0
                line = f.readline()    
        return mat


In [ ]:
# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)
from keras.layers import Input, Embedding, Flatten, Multiply, Dense
from keras import regularizers
from keras.models import Model
# from keras.optimizers import Adam

2.8.0


In [ ]:
# import tensorflow.keras.backend as K
# K.tensorflow_backend._get_available_gpus()

In [ ]:
# from keras.models import Sequential
# from keras.layers import Dense, Activation

# def sequential_NeuCF(num_users, num_items, latent_dim):
#     ## user_seq
#     user_seq = Sequential() # why (1,)
#     user_seq.add(Input(shape=(1,), dtype='int32', name='user_input')
#     user_seq.add(Embedding(input_dim=num_users, 
#                            output_dim=latent_dim, 
#                            embeddings_initializer='uniform',
#                            embeddings_regularizer=regularizers.l2(regs[0]),
#                            input_length=1)
#                 )
#     user_seq.add(Flatten())
    
#     ## item_seq
#     item_seq = Sequential()
#     item_seq.add(Input(shape=(1,), dtype='int32', name='item_input')
#     item_seq.add(Embedding(input_dim=num_items, 
#                            output_dim=latent_dim,
#                            embeddings_initializer='uniform',
#                            embeddings_regularizer=regularizers.l2(regs[1]),
#                            input_length=1)
#                 )
#     item_seq.add(Flatten())

#     ## NeuCF model
#     model = Sequential()
#     model.add(Multiply()([user_latent, item_latent]))

# model.add(layers.Dense(2, activation="relu"))

# model.summary()

In [ ]:
def get_model(num_users, num_items, latent_dim, regs=[0,0] ):
    user_input = Input(shape=(1,), dtype='int32', name='user_input') 
    item_input = Input(shape=(1,), dtype='int32', name='item_input')

    user_Embedding = Embedding(input_dim=num_users, output_dim=latent_dim, embeddings_initializer='uniform',
                               embeddings_regularizer=regularizers.l2(regs[0]),
                               input_length=1,
                               )

    item_Embedding = Embedding(input_dim=num_items, output_dim=latent_dim,
                               embeddings_initializer='uniform',
                               embeddings_regularizer=regularizers.l2(regs[1]),
                               input_length=1,
                               )

    user_latent = Flatten()(user_Embedding(user_input))
    item_latent = Flatten()(item_Embedding(item_input)) ##why (class's params)(args)

    predict_vertor = Multiply()([user_latent, item_latent])

    prediction = Dense(units=1,
                       activation='sigmoid',
                       kernel_initializer='lecun_uniform',
                       name = 'prediction', 
                       )(predict_vertor)
    model = Model(inputs=[user_input, item_input], outputs=prediction)
    return model 


dataset = Dataset(mvlens_dir + 'ml-1m')
train, testRatings, testNegatives = dataset.trainMatrix, dataset.testRatings, dataset.testNegatives
num_users, num_items = train.shape

alpha = 0.001

model = get_model(num_users=num_users, num_items=num_items, latent_dim=8, regs=[0,0])
model.compile(optimizer=keras.optimizers.Adam(learning_rate=alpha), loss="binary_crossentropy")


In [ ]:
model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 user_input (InputLayer)        [(None, 1)]          0           []                               
                                                                                                  
 item_input (InputLayer)        [(None, 1)]          0           []                               
                                                                                                  
 embedding (Embedding)          (None, 1, 8)         48320       ['user_input[0][0]']             
                                                                                                  
 embedding_1 (Embedding)        (None, 1, 8)         29648       ['item_input[0][0]']             
                                                                                              

In [ ]:
type(train)

scipy.sparse.dok.dok_matrix

In [ ]:
def get_train_instances(train, num_negatives):
    user_input, item_input, labels = [],[],[]
    num_users = train.shape[0]
    for (u, i) in train.keys():
        # positive instance
        user_input.append(u)
        item_input.append(i)
        labels.append(1)
        # negative instances
        for t in range(num_negatives):
            j = np.random.randint(num_items)
            while (u, j) in train:
                j = np.random.randint(num_items)
            user_input.append(u)
            item_input.append(j)
            labels.append(0)
    return user_input, item_input, labels


user_input, item_input, labels = get_train_instances(train, num_negatives=4)        

In [ ]:
labels[:4]

[1, 0, 0, 0]

In [ ]:
def eval_one_rating(idx):
    rating = _testRatings[idx]
    items = _testNegatives[idx]
    u = rating[0]
    gtItem = rating[1]
    items.append(gtItem)
    # Get prediction scores
    map_item_score = {}
    users = np.full(len(items), u, dtype = 'int32')
    predictions = _model.predict([users, np.array(items)], 
                                 batch_size=100, verbose=0)
    for i in range(len(items)):
        item = items[i]
        map_item_score[item] = predictions[i]
    items.pop()
    
    # Evaluate top rank list
    ranklist = heapq.nlargest(_K, map_item_score, key=map_item_score.get)
    hr = getHitRatio(ranklist, gtItem)
    ndcg = getNDCG(ranklist, gtItem)
    return (hr, ndcg)

def getHitRatio(ranklist, gtItem):
    for item in ranklist:
        if item == gtItem:
            return 1
    return 0

def getNDCG(ranklist, gtItem):
    for i in range(len(ranklist)):
        item = ranklist[i]
        if item == gtItem:
            return math.log(2) / math.log(i+2)
    return 0


def evaluate_model(model, testRatings, testNegatives, K, num_thread):
    """
    Evaluate the performance (Hit_Ratio, NDCG) of top-K recommendation
    Return: score of each test rating.
    """
    global _model
    global _testRatings
    global _testNegatives
    global _K
    _model = model
    _testRatings = testRatings
    _testNegatives = testNegatives
    _K = K
        
    hits, ndcgs = [],[]
    if(num_thread > 1): # Multi-thread
        pool = multiprocessing.Pool(processes=num_thread)
        res = pool.map(eval_one_rating, range(len(_testRatings)))
        pool.close()
        pool.join()
        hits = [r[0] for r in res]
        ndcgs = [r[1] for r in res]
        return (hits, ndcgs)
    # Single thread
    for idx in range(len(_testRatings)):
        (hr,ndcg) = eval_one_rating(idx)
        hits.append(hr)
        ndcgs.append(ndcg)      
    return (hits, ndcgs)


In [ ]:
checkpoint_path = mvlens_dir + "gmf_on_gpu.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

batch_size = 256
num_epochs = 20

topK = 10
evaluation_threads = 1 #mp.cpu_count()

for ep in range(num_epochs):

    # Train the model with the new callback
    history = model.fit(x=[np.array(user_input), np.array(item_input)], 
                    y=np.array(labels),  
                    epochs=1,
                    batch_size=batch_size, verbose=0, shuffle=True,
                    #   validation_data=(test_images, test_labels),
                    callbacks=[cp_callback]
                    )  # Pass callback to training

                        # This may generate warnings related to saving the state of the optimizer.
                        # These warnings (and similar warnings throughout this notebook)
                        # are in place to discourage outdated usage, and can be ignored.



    hits, ndcgs = evaluate_model(model, testRatings, testNegatives, topK, evaluation_threads)
    hr, ndcg = np.array(hits).mean(), np.array(ndcgs).mean()
    loss = history.history['loss'][0]

    print('Iter {}: HR = {:.4f}, NDCG = {:.4f}, Loss = {}\t '.format(ep+1, hr, ndcg, loss))



Epoch 1: saving model to /content/drive/MyDrive/data/ml-1m/gmf_on_gpu.ckpt
Iter 1: HR = 0.5242, NDCG = 0.2956, Loss = 0.36043569445610046	 

Epoch 1: saving model to /content/drive/MyDrive/data/ml-1m/gmf_on_gpu.ckpt
Iter 2: HR = 0.5786, NDCG = 0.3279, Loss = 0.3011660575866699	 

Epoch 1: saving model to /content/drive/MyDrive/data/ml-1m/gmf_on_gpu.ckpt
Iter 3: HR = 0.5987, NDCG = 0.3418, Loss = 0.2858373820781708	 

Epoch 1: saving model to /content/drive/MyDrive/data/ml-1m/gmf_on_gpu.ckpt
Iter 4: HR = 0.6089, NDCG = 0.3468, Loss = 0.27739977836608887	 

Epoch 1: saving model to /content/drive/MyDrive/data/ml-1m/gmf_on_gpu.ckpt
Iter 5: HR = 0.6147, NDCG = 0.3505, Loss = 0.27407434582710266	 

Epoch 1: saving model to /content/drive/MyDrive/data/ml-1m/gmf_on_gpu.ckpt
Iter 6: HR = 0.6232, NDCG = 0.3560, Loss = 0.27229857444763184	 

Epoch 1: saving model to /content/drive/MyDrive/data/ml-1m/gmf_on_gpu.ckpt
Iter 7: HR = 0.6185, NDCG = 0.3540, Loss = 0.2711341083049774	 

Epoch 1: saving